# Overview of the Artifact Processing Code

This code helps you process text artifacts using OpenAI’s API. It includes an example that summarizes `.txt` files, handling errors and retries automatically.

## What You Should Do

1. **Run the Code**: Ensure the code works in your environment:
   - Place `.txt` files in the `code_and_data_quick_start` folder.
   - Run the script and confirm the output matches the description.

2. **Modify for Your Experiments**:
   - Replace the example function with one tailored to your experiment.
   - Adapt the prompts, file types, or output format as needed.

## Support Resources

If you have questions or need help:
- **ChatGPT**: Ask for assistance with adapting or debugging your code.
- **Your Classmates**: Collaborate with teammates to brainstorm and troubleshoot.
- **Team 6**: Reach out to the coding support team for technical help.
- **Your Instructor (Prof. Allen)**: I’m here to guide you and answer your questions.

Take it step by step, and remember: experimenting is part of the process. You've got this!cess. You've got this!

In [13]:
import os
import time
from queue import Queue
from openai import OpenAI

# Initialize the OpenAI client
api_key_file_path = "Team 06 API Key.txt"
def read_and_validate_api_key(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"API key file '{file_path}' not found.")
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    if len(lines) < 2:
        raise ValueError(f"API key file '{file_path}' must have at least two lines.")
    raw_key = lines[1].strip()
    if not raw_key.startswith("sk-svcacct-"):
        raise ValueError(f"Invalid API key format in file '{file_path}'. Key must start with 'sk-svcacct-'.")
    return raw_key

try:
    api_key = read_and_validate_api_key(api_key_file_path)
    client = OpenAI(api_key=api_key)
    print(f"Validated API key: {api_key[:8]}...{api_key[-6:]}")
except Exception as e:
    print(f"Error: {e}")

# Function to generate a response using OpenAI API
def generate_response(client, system_role, user_prompt, model="gpt-4o-mini"):
    try:
        completion = client.chat.completions.create(
            model=model,
            messages=[
                { "role": "system", "content": system_role },
                { "role": "user", "content": user_prompt },
            ]
        )
        return completion.choices[0].message.content
    except Exception as e:
        raise RuntimeError(f"Error connecting to OpenAI API: {e}")

# Function to summarize a future history
def summarize_future_history(client, file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            file_content = f.read()
        system_role = (
            "You are a helpful assistant. Your task is to read future histories written by students."
            " A 'future history' is a narrative written in past tense about events that the writer imagines will happen in the future."
        )
        user_prompt = f"Summarize the following future history in one sentence:\n\n{file_content}"
        response = generate_response(client, system_role, user_prompt, 'gpt-4o')
        return response.strip()
    except Exception as e:
        raise RuntimeError(f"Error summarizing file '{file_path}': {e}")

# Process all .txt files in the specified folder
artifacts_file_path = 'code_and_data_quick_start/'
file_queue = Queue()
for file_name in os.listdir(artifacts_file_path):
    file_path = os.path.join(artifacts_file_path, file_name)
    if os.path.isfile(file_path) and file_name.lower().endswith(".txt"):
        file_queue.put(file_path)

max_retries = 3
retry_delays = [1, 2, 4]
attempts = {}

while not file_queue.empty():
    file_path = file_queue.get()
    attempts[file_path] = attempts.get(file_path, 0) + 1
    try:
        summary = summarize_future_history(client, file_path)
        print(f"Summarizing file: {file_path}")
        print("Summary:")
        print(summary)
        print("\n" + "-" * 50 + "\n")
    except Exception as error:
        print(f"Error processing file '{file_path}': {error}")
        if attempts[file_path] <= max_retries:
            print(f"Retrying file: {file_path} (attempt {attempts[file_path]})")
            file_queue.put(file_path)
            time.sleep(retry_delays[min(attempts[file_path] - 1, len(retry_delays) - 1)])
        else:
            print(f"Max retries reached for file: {file_path}. Skipping.")

Validated API key: sk-svcac...4pX4gA


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'code_and_data_quick_start/'

## Rubric Information

1. **Add `rubric-history-1.txt` and `instructions-history.txt` to the `code_and_data_quick_start` folder**.
2. **The program will read the instructions and essays automatically**.
3. **The program will ask ChatGPT to evaluate them based on the rubric file automatically**.
4. **Ensure `rubrics.txt` is included in the zipped folder `code_and_data_quick_start`**.

In [16]:
# Function to evaluate essays based on rubric
def evaluate_essays(client, rubric_path, instructions_path, essays_path):
    try:
        with open(rubric_path, "r", encoding="utf-8") as f:
            rubric_content = f.read()
        with open(instructions_path, "r", encoding="utf-8") as f:
            instructions_content = f.read()
        with open(essays_path, "r", encoding="utf-8") as f:
            essays_content = f.read()
        
        system_role = "You are an evaluator. Use the provided rubric to evaluate the following essays."
        user_prompt = f"Rubric:\n{rubric_content}\n\nInstructions:\n{instructions_content}\n\nEssays:\n{essays_content}"
        
        response = generate_response(client, system_role, user_prompt, 'gpt-4o')
        return response.strip()
    except Exception as e:
        raise RuntimeError(f"Error evaluating essays: {e}")

# Evaluate essays
try:
    rubric_path = "code_and_data_quick_start/rubric-history-1.txt"
    instructions_path = "code_and_data_quick_start/instructions-history.txt"
    essays_path = "code_and_data_quick_start/essays.txt"
    evaluation = evaluate_essays(client, rubric_path, instructions_path, essays_path)
    print("Evaluation Result:")
    print(evaluation)
except Exception as error:
    print(f"Error: {error}")

Error: Error evaluating essays: [Errno 2] No such file or directory: 'code_and_data_quick_start/rubric-history-1.txt'


## Explanation for Students

1. **Add `rubrics.txt` into the zipped folder `code_and_data_quick_start`**.
2. **Place your API key file, rubric file, instructions file, and essays file in the `code_and_data_quick_start` folder**.
3. **Run the notebook to process and evaluate the essays based on the rubric**.